## Import the libraries

In [21]:
import pydicom
import numpy as np
from PIL import Image

In [8]:
dicom_file=pydicom.dcmread("data/slice.IMA")

In [9]:
dicom_file

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 192
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.1.4.45508.30000019041301312660900000455
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.3.12.2.1107.5.1.4
(0002,0013) Implementation Version Name         SH: 'SIEMENS_S5VB42A'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'LOCALIZER', 'CT_SOM5 TOP']
(0008,0016) SOP Class UID                       UI: CT Image Storage
(0008,0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.1.4.45508.30000019041301312660900000455
(0008,0020) Study Date               

In [10]:
dicom_file.Manufacturer

'SIEMENS'

In [11]:
dicom_array = dicom_file.pixel_array

In [10]:
dicom_array.shape

(512, 512)

## Show Image

In [12]:
type(dicom_array)

numpy.ndarray

In [13]:
np.unique(dicom_array)

array([   0,  782,  786,  789,  792,  799,  800,  802,  805,  806,  813,
        814,  820,  825,  829,  830,  832,  833,  835,  836,  837,  838,
        840,  841,  842,  843,  844,  845,  847,  848,  849,  850,  852,
        853,  854,  855,  856,  857,  858,  859,  860,  861,  862,  863,
        864,  865,  866,  867,  868,  869,  870,  871,  872,  873,  874,
        875,  876,  877,  878,  879,  880,  881,  882,  883,  884,  885,
        886,  887,  888,  889,  890,  891,  892,  893,  894,  895,  896,
        897,  898,  899,  900,  901,  902,  903,  904,  905,  906,  907,
        908,  909,  910,  911,  912,  913,  914,  915,  916,  917,  918,
        919,  920,  921,  922,  923,  924,  925,  926,  927,  928,  929,
        930,  931,  932,  933,  934,  935,  936,  937,  938,  939,  940,
        941,  942,  943,  944,  945,  946,  947,  948,  949,  950,  951,
        952,  953,  954,  955,  956,  957,  958,  959,  960,  961,  962,
        963,  964,  965,  966,  967,  968,  969,  9

In [45]:
float_dicom_array = dicom_array.astype(float)
positive_dicom_array = np.maximum(float_dicom_array,0)

In [48]:
normalized_dicom_array = positive_dicom_array/positive_dicom_array.max()
normalized_dicom_array*=255.0
uint8_image=np.uint8(normalized_dicom_array)

In [22]:
# np.unique(normalized_dicom_array)

In [49]:
pillow_image = Image.fromarray(uint8_image)
pillow_image.show()

## Show Image 2

In [56]:
hounsfield_min = np.min(dicom_array)
hounsfield_max = np.max(dicom_array)
hounsfield_range = hounsfield_max - hounsfield_min

In [57]:
dicom_array[dicom_array < hounsfield_min] = hounsfield_min
dicom_array[dicom_array > hounsfield_max] = hounsfield_max
normalized_array_2 = (dicom_array - hounsfield_min) / hounsfield_range
normalized_array_2 *= 255
uint8_image_2 = np.uint8(normalized_array_2)

In [58]:
pillow_image = Image.fromarray(uint8_image_2)
pillow_image.show()